<a href="https://colab.research.google.com/github/navi004/Deep-Learning-CSE4037-/blob/main/DL_Lab_8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LSTM and GRU fusion - Lab A-5b
* Naveen N (22MIA1049)  

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/IMDB Dataset.csv")
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
t = Tokenizer(num_words=5000)

t.fit_on_texts(df['review'])

x = t.texts_to_sequences(df['review'])

max_length = 100

xp = pad_sequences(x, maxlen=max_length, padding = 'post')

In [ ]:
xp

array([[ 954,  183,    1, ...,  125, 4103,  486],
       [  29,    1, 2294, ..., 1977,   69,  221],
       [1026,  220,  230, ...,   63,   16,  350],
       ...,
       [   9,  200,   26, ..., 1957,    2,    2],
       [  16,   47, 4834, ...,   67,  739,   42],
       [  49,   14,   47, ...,  794,   11,   17]], dtype=int32)

In [ ]:
y = np.array(df['sentiment'].map({'positive' : 1, 'negative' : 0}))
y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(xp, y, test_size=0.2, random_state=42)

print(x_train.shape)

(40000, 100)


### LSTM and GRU fusion Model

In [ ]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=32, input_length=max_length),  # Word embeddings
    Bidirectional(LSTM(32, return_sequences=True)),  # BiLSTM to capture long dependencies
    Bidirectional(GRU(16, return_sequences=False)),  # BiGRU for efficiency
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 156s 119ms/step - accuracy: 0.7603 - loss: 0.4674 - val_accuracy: 0.8586 - val_loss: 0.3404
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 201s 118ms/step - accuracy: 0.8904 - loss: 0.2701 - val_accuracy: 0.8651 - val_loss: 0.3035
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 146s 117ms/step - accuracy: 0.9169 - loss: 0.2105 - val_accuracy: 0.8579 - val_loss: 0.3254
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 148s 118ms/step - accuracy: 0.9349 - loss: 0.1707 - val_accuracy: 0.8627 - val_loss: 0.3504
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 148s 118ms/step - accuracy: 0.9551 - loss: 0.1215 - val_accuracy: 0.8591 - val_loss: 0.4057


### Accuracy and loss

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (32, 100, 32)               │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (32, 100, 64)               │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (32, 32)                    │           7,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (32, 1)                     │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 553,637 (2.11 MB)

 Trainable params: 184,545 (720.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 369,092 (1.41 MB)

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.8599 - loss: 0.3989
Test Loss: 0.4057365655899048, Test Accuracy: 0.8590999841690063


### Testing with new record

In [ ]:
txt = 'The movie is so boring and the direction is bad'

txt = t.texts_to_sequences([txt])

txtP = pad_sequences(txt, maxlen=max_length, padding='post')

pred = model.predict(txtP)

print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 706ms/step
[[0.00729743]]


In [ ]:
print("Sentiment : Positive" if pred[0][0] > 0.5 else "Sentiment : Negative")

Sentiment : Negative
